In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 282 ms (started: 2021-09-05 13:56:47 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb */*.ipynb

git remote -v

git commit -m '更新 #5 Sept 05, 2021'

#git push origin master
git push

保存和加载多个模型有助于重用您之前训练过的模型。

# 介绍

保存由多个 torch.nn.Modules 组成的模型时，例如 GAN、序列到序列模型或模型集合，您必须保存每个模型的 state_dict 和相应优化器的字典。 您还可以通过简单地将它们附加到字典来保存可能有助于您恢复训练的任何其他项目。 要加载模型，首先初始化模型和优化器，然后使用 torch.load() 在本地加载字典。 从这里，您可以通过简单地按预期查询字典来轻松访问保存的项目。 在这个秘籍中，我们将演示如何使用 PyTorch 将多个模型保存到一个文件中。

# 步骤
* 导入所有必要的库以加载我们的数据
* 定义并初始化神经网络
* 初始化优化器
* 保存多个模型
* 加载多个模型

1. 导入必要的库来加载我们的数据

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

time: 711 ms (started: 2021-09-05 13:59:08 +08:00)


2. 定义并初始化神经网络

例如，我们将创建一个用于训练图像的神经网络。 要了解更多信息，请参阅定义神经网络食谱。 为模型构建两个变量以最终保存。

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

netA = Net()
netB = Net()

time: 28.4 ms (started: 2021-09-05 13:59:43 +08:00)


3. 初始化优化器

我们将使用带有动量的 SGD 为我们创建的每个模型构建优化器。

In [4]:
optimizerA = optim.SGD(netA.parameters(), lr=0.001, momentum=0.9)
optimizerB = optim.SGD(netB.parameters(), lr=0.001, momentum=0.9)

time: 1.43 ms (started: 2021-09-05 14:00:15 +08:00)


4. 保存多个模型

收集所有相关信息并构建您的字典。

In [5]:
# Specify a path to save to
PATH = "model.pt"

torch.save({
            'modelA_state_dict': netA.state_dict(),
            'modelB_state_dict': netB.state_dict(),
            'optimizerA_state_dict': optimizerA.state_dict(),
            'optimizerB_state_dict': optimizerB.state_dict(),
            }, PATH)

time: 6.74 ms (started: 2021-09-05 14:00:56 +08:00)


5. 加载多个模型

请记住首先初始化模型和优化器，然后在本地加载字典。

In [ ]:
modelA = Net()
modelB = Net()
optimModelA = optim.SGD(modelA.parameters(), lr=0.001, momentum=0.9)
optimModelB = optim.SGD(modelB.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['modelA_state_dict'])
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

modelA.eval()
modelB.eval()
# - or -
modelA.train()
modelB.train()

在运行推理之前，您必须调用 model.eval() 将 dropout 和批量归一化层设置为评估模式。 不这样做会产生不一致的推理结果。

如果您希望恢复训练，请调用 model.train() 以确保这些层处于训练模式。

恭喜！ 您已在 PyTorch 中成功保存和加载多个模型。